# 5.Test

### Import libraries
The required libraries for this notebook are pandas, sklearn, copy, numpy, pickle and matplotlib.




In [3]:
# import libraries
import numpy as np
import pandas as pd
import csv
import cv2
import os
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import joblib
import tensorflow as tf
from sklearn import metrics

import openpyxl

### Load the data
We are going to use the Brain Tumors dataset 

In [2]:
data=pd.read_csv('./test/label.csv')
df0=pd.DataFrame(data)

In [3]:
for root, dirs, files in os.walk('./test/image'):
    for d in dirs:
        print(d) 
    i=0
    Data=[]
    for file in files:  
        img_path = root+'/'+file
        img = cv2.imread(img_path,0)
        image_data = img.flatten()
        Data.append(image_data)
    Data_Array = np.array(Data)
    df = pd.DataFrame(Data_Array)
    df2 = pd.concat([df0,df],axis=1)
    print(df2.shape)


(200, 262146)


### Dimension Redusion

In [4]:
pca = joblib.load('./model/pca.m')
data_reduced=pca.transform(df)
print(data_reduced.shape)

(200, 839)


In [5]:
np.save('test_reduced',data_reduced)

In [6]:
target=[]
for tumor in df0.label:
    if tumor == 'no_tumor':
        target.append(0)
    else:
        target.append(1)
target = np.array(target)

In [7]:
counts=Counter(target)
print(counts)# show the distribution of sample

Counter({1: 163, 0: 37})


In [8]:
test =data_reduced
scaler = MinMaxScaler() 
test1 = scaler.fit_transform(data_reduced)

scaler = StandardScaler()
test2 = scaler.fit_transform(data_reduced)

### Binary classification

In [9]:
log = joblib.load('./model/log_binary.m')
y_pred_log=log.predict(test2)
score_log=metrics.accuracy_score(target,y_pred_log)
print('Logistic Regression Accuracy on test set:',score_log)

Logistic Regression Accuracy on test set: 0.905


In [10]:
knn = joblib.load('./model/knn_binary.m')
y_pred_KNN=knn.predict(test)
score_KNN=metrics.accuracy_score(target,y_pred_KNN)
print('K-Nearest Neighbours Accuracy on test set:',score_KNN)

K-Nearest Neighbours Accuracy on test set: 0.97


In [11]:
SVM = joblib.load('./model/SVM_binary.m')
y_pred_SVM=SVM.predict(test)
score_SVM=metrics.accuracy_score(target,y_pred_SVM)
print('Support Vector Machine Accuracy on test set:',score_SVM)

Support Vector Machine Accuracy on test set: 0.925


In [12]:
scaler = StandardScaler()
test3 = scaler.fit_transform(df)

In [13]:
RF = joblib.load('./model/RandomForest_binary.m')
y_pred_RandomForest=RF.predict(test3)
score_RandomForest=metrics.accuracy_score(target,y_pred_RandomForest)
print('Random Forest Accuracy on test set:',score_RandomForest)

Random Forest Accuracy on test set: 0.915


### Multiclass

    0 represents no tumor     
    1 represents meningioma_tumor
    2 represents glioma_tumor
    3 represents pituitary_tumor

In [14]:
Target=[]
for tumor in df0.label:
    if tumor == 'no_tumor':
        Target.append(0)
    elif tumor == 'meningioma_tumor':
        Target.append(1)
    elif tumor == 'glioma_tumor':
        Target.append(2)
    elif tumor == 'pituitary_tumor':
        Target.append(3)
Target = np.array(Target)
#print(Target)

In [15]:
SVM_multiple = joblib.load('./model/SVM_multiple.m')
y_pred_SVM_multiple =SVM_multiple .predict(test)
score_SVM_multiple=metrics.accuracy_score(Target,y_pred_SVM_multiple )
print('Support Vector Machine Accuracy on test set:',score_SVM_multiple)

Support Vector Machine Accuracy on test set: 0.84


In [68]:
MLP =tf.keras.models.load_model('./model/MLP.h5')

In [69]:
y_pred_MLP =MLP.predict_classes(test2)
score_MLP=metrics.accuracy_score(Target,y_pred_MLP )
print('Multi-layer Perceptron classifier Accuracy on test set:',score_MLP)

Multi-layer Perceptron classifier Accuracy on test set: 0.825


In [70]:
name=['Logistic Regression','K-Nearest Neighbours','Random Forest','Support Vector Machine','Support Vector Machine','Multi-layer Perceptron classifier']
Score=[score_log,score_KNN,score_RandomForest,score_SVM, score_SVM_multiple,score_MLP]
Data = {'Method': name, 'Percentage': Score}
Table = pd.DataFrame(data=Data,index=[['binary','binary','binary','binary','multiclass','multiclass'],[1,2,3,4,1,2]])
Table

Method  Percentage
binary     1                Logistic Regression       0.905
           2               K-Nearest Neighbours       0.970
           3                      Random Forest       0.915
           4             Support Vector Machine       0.925
multiclass 1             Support Vector Machine       0.840
           2  Multi-layer Perceptron classifier       0.825

In [71]:
name=['Logistic Regression','K-Nearest Neighbours','Support Vector Machine','Multi-layer Perceptron classifier']
Score=[score_log,score_KNN, score_SVM_multiple,score_MLP]
Data = {'Method': name, 'Accuracy': Score}
Tasks =['binary','binary','multiclass','multiclass']
Number = [1,2,1,2]
Data = {'Method': name, 'Accuracy': Score, 'Tasks':Tasks, 'Number':Number}
table = pd.DataFrame(data=Data)
Table = table.set_index(['Tasks','Number'])
Table.to_excel('./Diagram/Table1.xlsx')
Table

Method  Accuracy
Tasks      Number                                             
binary     1                     Logistic Regression     0.905
           2                    K-Nearest Neighbours     0.970
multiclass 1                  Support Vector Machine     0.840
           2       Multi-layer Perceptron classifier     0.825

In [72]:
Table.to_csv('results.csv')

In [73]:
# table = pd.read_csv('results.csv')
# Table = table.set_index(['Tasks','Number'])

In [74]:
test_Proportion=Counter(counts)
print(test_Proportion)

Counter({1: 163, 0: 37})


In [75]:
logRegr_Proportion=Counter(y_pred_log)
print(logRegr_Proportion)

Counter({1: 168, 0: 32})


In [76]:
KNN_Proportion=Counter( y_pred_KNN)
print(KNN_Proportion)

Counter({1: 163, 0: 37})


In [77]:
SVM_Proportion=Counter(y_pred_SVM)
print(SVM_Proportion)

Counter({1: 174, 0: 26})


In [78]:
compare={'target':test_Proportion,'Log':logRegr_Proportion,'KNN':KNN_Proportion,'SVM':SVM_Proportion}
com_table=pd.DataFrame(compare)
com_table.to_excel('./Diagram/Table2.xlsx')
com_table

,target,Log,KNN,SVM
1,163,168,163,174
0,37,32,37,26


0 represents No tumor，1 represents Tumor

In [79]:
Test_Proportion=Counter(Target)
print(Test_Proportion)

Counter({1: 68, 3: 52, 2: 43, 0: 37})


In [80]:
SVM_Proportion_multiple=Counter(y_pred_SVM_multiple)
print(SVM_Proportion_multiple)

Counter({3: 63, 1: 59, 2: 46, 0: 32})


In [81]:
MLP_Proportion=Counter(y_pred_MLP)
print(MLP_Proportion)

Counter({1: 71, 3: 59, 2: 40, 0: 30})


In [82]:
Compare={'target':Test_Proportion,'SVM':SVM_Proportion_multiple,'MLP':MLP_Proportion}
Com_table=pd.DataFrame(Compare)
Com_table.to_excel('./Diagram/Table3.xlsx')
Com_table

,target,SVM,MLP
1,68,59,71
2,43,46,40
3,52,63,59
0,37,32,30


    0 represents no tumor     
    1 represents meningioma_tumor
    2 represents glioma_tumor
    3 represents pituitary_tumor